In [ ]:
import numpy as np
import xarray as xr
import fiona
import math
import rasterio.features
import datacube 
import warnings
from datacube.storage import masking
from datacube.utils import geometry
from datacube.helpers import ga_pq_fuser
import folium
from pyproj import Proj, transform
import geopandas as gpd
import ipywidgets
import matplotlib as mpl
from ipyleaflet import Map, Marker, Popup, GeoJSON, basemaps

In [ ]:
def calculate_anomalies(veg_index, from_shape, shp_fpath,
                        time, season, region, lat, lon, buffer, chunk_size):
    
    #error messaging for important inputs
    if veg_index not in ('msavi', 'ndvi'):
        raise ValueError("Veg_index must be either 'msavi' or 'ndvi'")
    

    if from_shape:
        if len(fiona.open(shp_fpath)) > 1:
            warnings.warn("This script can only accept shapefiles with a single polygon feature; "
                          "seasonal anomalies will be calculated for the extent of the "
                          "first geometry in the shapefile only.")
        
    #connect to datacube
    dc = datacube.Datacube(app='load_clearlandsat')
    
    #get data from shapefile extent and mask
    if from_shape:
        print("extracting data based on shapefile extent")
        
        with fiona.open(shp_fpath) as input:
            crs = geometry.CRS(input.crs_wkt)
        
        feat = fiona.open(shp_fpath)[0]
        first_geom = feat['geometry']
        geom = geometry.Geometry(first_geom, crs=crs)

        query = {'geopolygon': geom,
                 'time': time}
        
        ds = load_landsat(dc=dc, query=query, sensors=['ls5','ls7','ls8'], 
                           bands_of_interest=['nir', 'red'], lazy_load=True,
                           dask_chunks = {'x': chunk_size, 'y': chunk_size})
        
        mask = rasterio.features.geometry_mask([geom.to_crs(ds.geobox.crs)for geoms in [geom]],
                                       out_shape=ds.geobox.shape,
                                       transform=ds.geobox.affine,
                                       all_touched=False,
                                       invert=False)
        
        mask_xr = xr.DataArray(mask, dims = ('y','x'))
        ds = ds.where(mask_xr==False)
        
    else: 
        print('Extracting data based on lat, lon coords')
        # todo - change to bounding box
        query = {'lon': (lon - buffer, lon + buffer),
                 'lat': (lat - buffer, lat + buffer),
                 'time': time}
        
        ds = load_landsat(dc=dc, query=query, sensors=['ls5','ls7','ls8'], 
                           bands_of_interest=['nir', 'red'], lazy_load=True,
                           dask_chunks = {'x': chunk_size, 'y': chunk_size})

    
    print('calculating vegetation indice')
    
    if veg_index == 'msavi':
        # calculate the seasonal mean of MSAVI
        nir = ds.nir / 10000
        red = ds.red / 10000
        vegIndex = (2*nir+1-((2*nir+1)**2 - 8*(nir-red))**0.5)/2
        vegIndex = vegIndex.astype('float32')
        vegIndex = vegIndex.mean('time').rename(veg_index+'_mean')
    
    if veg_index == 'ndvi':
        vegIndex = (ds.nir - ds.red) / (ds.nir + ds.red)
        vegIndex = vegIndex.astype('float32')
        vegIndex = vegIndex.mean('time').rename(veg_index+'_mean')
        
    #get the bounding coords of the input ds to help with indexing the climatology
    #jbw - shouldn't be an issue if lat lon or bounding boxes?
    xmin, xmax = vegIndex.x.values[0], vegIndex.x.values[-1]
    ymin,ymax = vegIndex.y.values[0], vegIndex.y.values[-1]
    x_slice = [i+0.5 for i in range(int(xmin),int(xmax+25),25)]
    y_slice = [i-0.5 for i in range(int(ymin),int(ymax-25),-25)]
    
    #index the climatology dataset to the location of our AOI
    #fixme - what are the filenames going to be?
    print('Opening mean climatology for: NSW ' + veg_index +' '+ season)
    mean_climatology = xr.open_rasterio('results/NSW_' + veg_index + "_clim_mean_"+ season +'_mosaic.tif',
                                  chunks=chunk_size).sel(x=x_slice, y=y_slice, method='nearest').squeeze()
        print('Opening standard deviation climatology for: NSW ' + veg_index +' '+ season)
    stdev_climatology = xr.open_rasterio('results/NSW_' + veg_index + "_clim_stdev_"+ season +'_mosaic.tif',
                                  chunks=chunk_size).sel(x=x_slice, y=y_slice, method='nearest')
    
    #test if the arrays match before we calculate the anomalies
    np.testing.assert_allclose(vegIndex.x.values, climatology.x.values,
                              err_msg="The X coordinates on the AOI dataset do not match "
                                      "the X coordinates on the climatology dataset. "
                                       "You're AOI may be beyond the extent of the pre-computed "
                                       "climatology dataset.")
    
    np.testing.assert_allclose(vegIndex.y.values, climatology.y.values,
                          err_msg="The Y coordinates on the AOI dataset do not match "
                                  "the Y coordinates on the climatology dataset. "
                                   "You're AOI may be beyond the extent of the pre-computed "
                                   "climatology dataset.")
    
    print('calculating anomalies')
    #calculate anomalies
    anomalies = (vegIndex - mean_climatology)/stdev_climatology

    
    #add back metadata
    anomalies = anomalies.rename(veg_index + '_anomalies').to_dataset()
    anomalies.attrs = ds.attrs
    anomalies.attrs['units'] = 1

    return anomalies
